## In this script we run a Costa Rica 2060 scenario, so convergence around 10000$ yearly disposable income or consumption per person, and see country specific implications and create figure 5 of the paper

#### Specifically we measure the decarbonization rates, so emission reductions rates in per capita terms for all countries specifically and under three distinct scenarios (i) countries above 10000 with degrowth, (ii) with steady state (iii) with continuous growth

In [2]:
from scenario_class import Scenario
from plots_class import Plots
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.cm as cm

scenario_params = {"end_year": 2060,
                   "income_goal": 10000, # in terms of 2017 USD PPP disposable household income per capita (the delineation between income and cons. exp is not necessarily clear)
                   "carbon_budget": 1150, # in terms of GtCO2 that is gigatons of CO2        
                   "hysteresis_tech_progress": 0.01, # this is a growth rate, so between 0 and 1, which only applies if tech_hysteresis_assumption is on
                   "gdp_assumption": "constant_ratio", # this is a string and either (1) constant_ratio or (2) model_ratio for details see the scenario_class.py file         
                   "pop_growth_assumption": "semi_log_model", # this is a string and either (1) UN_medium, (2) semi_log_model (3) semi_log_model_elasticity for details see the scenario_class.py file  
                   "tech_evolution_assumption": "plausible", # this is a string and either (1) plausible or (2) necessary for details see the scenario_class.py file
                   "tech_hysteresis_assumption": "off", # this is a string and either (1) on or (2) off for details see the scenario_class.py file 
                   "steady_state_high_income_assumption": "off", # this is a string and either (1) on or (2) off or (3) on_with_growth for details see the scenario_class.py file
            }

scenario_1 = Scenario(scenario_params)
#test = scenario_1.raw_data
scenario_1.compute_country_scenario_params()
scenario_1.run()

In [3]:
## Read out the country specific carbon emissions per capita results
for country in scenario_1.countries.values():
    print(country.code)

AGO
ALB
ARE
ARM
AUS
AUT
BDI
BEL
BEN
BFA
BGD
BGR
BIH
BLR
BOL
BRA
BTN
BWA
CAF
CAN
CHE
CHL
CHN
CIV
CMR
COD
COG
COL
COM
CPV
CRI
CYP
CZE
DEU
DNK
DOM
DZA
ECU
EGY
ESP
EST
ETH
FIN
FJI
FRA
FSM
GAB
GBR
GEO
GHA
GIN
GMB
GNB
GRC
GTM
HND
HRV
HTI
HUN
IDN
IND
IRL
IRN
IRQ
ISL
ISR
ITA
JPN
KAZ
KEN
KGZ
KIR
KOR
LAO
LBN
LBR
LCA
LKA
LSO
LTU
LUX
LVA
MAR
MDA
MDG
MDV
MEX
MHL
MKD
MLI
MLT
MMR
MNE
MNG
MOZ
MRT
MUS
MWI
MYS
NAM
NER
NGA
NIC
NLD
NOR
NRU
PAK
PAN
PER
PHL
POL
PRT
PRY
PSE
ROU
RUS
RWA
SDN
SEN
SLB
SLE
SLV
SRB
SSD
STP
SVK
SVN
SWE
SWZ
SYC
TCD
TGO
THA
TJK
TLS
TON
TUN
TUR
TZA
UGA
UKR
URY
USA
VNM
VUT
WSM
XKX
ZAF
ZMB
ZWE


In [4]:
import pandas as pd
# Assuming scenario_1.countries is a list-like object with each element having a .name and .carbon_emissions_pc_trajectory attribute
data = {country.code: country.carbon_emissions_pc_trajectory for country in scenario_1.countries.values()}
df_emissions = pd.DataFrame(data).T  # Transpose to have countries as rows and years as columns


def calculate_cagr(ev, bv, n):
    """Calculate the Compound Annual Growth Rate (CAGR)."""
    return (ev / bv) ** (1 / n) - 1

# Calculate CAGR for each country
cagr_data = {}
for country, emissions in df_emissions.iterrows():
    first_year, last_year = emissions.first_valid_index(), emissions.last_valid_index()
    bv = emissions[first_year]
    ev = emissions[last_year]
    n = int(last_year) - int(first_year)
    cagr = calculate_cagr(ev, bv, n)
    cagr_data[country] = cagr

df_cagr = pd.DataFrame.from_dict(cagr_data, orient='index', columns=['CAGR'])

print(df_cagr)

         CAGR
AGO  0.029235
ALB -0.007109
ARE -0.051171
ARM  0.014766
AUS -0.047082
..        ...
WSM  0.021896
XKX  0.003976
ZAF  0.005172
ZMB  0.072618
ZWE  0.036260

[150 rows x 1 columns]
